# CSV to ee.FeatureCollection
This script convers the PM-2.5 ground station data (in a csv) to an ee.FeatureCollection for upload to Google Earth Engine.

In [ ]:
# Accesses Earth Engine API
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Open file using csv library
filename = 'PM_TAW_AY_2b.csv'

import csv
with open(filename, 'r', newline='') as csvfile:
  reader = csv.DictReader(csvfile)

  # parse the dataframe for date, hour, site name, and raw value
  # convert components to EE objects
  pt_num, xy, sites, rawvalues, datetimes = [], [], [], [], []
  unique_dates = {}

  for row in reader:

    datetimes.append(ee.Date(row['datetime']).millis())
    rawvalues.append(ee.Number(float(row['rawvalue'])))
    xy.append([ee.Number(float(row['longitude'])), ee.Number(float(row['latitude']))])
    sites.append(ee.String(row['sitename']))
    pt_num.append(ee.Number(float(row['num'])))

In [ ]:
# convert to ee.Lists
ee_pt_num = ee.List(pt_num)
ee_xy = ee.List(xy)
ee_sites = ee.List(sites)
ee_rawvalues = ee.List(rawvalues)
ee_datetimes = ee.List(datetimes)

In [ ]:
# combine into Earth Engine feature collection
def combine(number):
    point = ee_xy.get(number)
    datetime = ee_datetimes.get(number)
    station = ee_sites.get(number)
    pm25 = ee_rawvalues.get(number)
    return ee.Feature(ee.Geometry.Point(point),
     {'station': station, 'pm25': pm25}).set('system:time_start', datetime)

data = ee.FeatureCollection(ee_pt_num.map(combine))

In [ ]:
# export to Google Earth Engine
ee.batch.Export.table.toAsset(data, 'TAW_AY_2b', 'users/aspenjkmorgan/PM_FC/TAW_AY_2b').start()